# 推定

In [16]:
import numpy as np
import pandas as pd
import scipy as sp
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()
%precision 3
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

## 点推定

- 母数をある1つの値として指定する推定方法
- 母数を推定する場合、標本平均を推定量として使う


In [17]:
fish = pd.read_csv('../data/3-7-1-fish_length.csv')['length']
# 母平均の点推定値としては、標本平均を利用
mu = sp.mean(fish)
# 母分散の点推定値としては、不偏分散を利用
sigma_2 = sp.var(fish, ddof=1)
print(f'標本平均: {mu}, 不偏分散: {sigma_2}')

標本平均: 4.187039324504523, 不偏分散: 0.6803017080832623


## 区間推定

* 推定値に幅を持たせた推定方法のことを指す
* 幅を持たせることで、推定誤差を加味することが可能になる
  * サンプルサイズが大きくなることで、区間推定の幅は小さくなる(=大数の法則)
* 信頼係数: 区間推定の幅における信頼度合いを、確率で表現したもの
* 信頼区間: ある信頼係数を満たす区間
* 信頼限界: 信頼区間の下限値・上限値のこと

$$ t = \frac{\hat{\mu}-\mu}{\hat{\sigma}/\sqrt{N}}
 = \frac{標本平均-母平均}{標準誤差} $$

信頼区間を95%とするなら、t分布における2.5%点と97.5%点を計算することで求めることができる。
t分布に従う変数が、2.5%点と97.5%点の区間に入る確率が95%ということになる


In [18]:
df = len(fish) -1
sigma = sp.std(fish, ddof=1)
se = sigma / sp.sqrt(len(fish))

t_975 = stats.t.ppf(q=0.975, df=df)
t_025 = stats.t.ppf(q=0.025, df=df)
upper = -(t_025 * se) + mu
lower = -(t_975 * se) + mu
print(lower, upper)
interval = stats.t.interval(
    alpha=0.95, df=df, loc=mu, scale=se
)
interval
# 3.597 ~ 4.777 が95%信頼区間

3.597010056835825 4.777068592173222


(3.597, 4.777)

## 信頼区間の幅を決める要素

- 信頼区間の幅が広い(=分散が大きい)というのは、「真の母平均がどこに位置しているのかわからない」ということを指す
- サンプルサイズが大きくなることで、標本平均を信頼できるようになるため、信頼区間は狭くなる

In [21]:
se2 = (sigma*10)/sp.sqrt(len(fish))
print(stats.t.interval(
    alpha=0.95, df=df, loc=mu, scale=se2
))

df2 = (len(fish)*10)-1
se3 = sigma/sp.sqrt(len(fish)*10)
print(stats.t.interval(
    alpha=0.95, df=df2, loc=mu, scale=se3
))

(-1.7132533521824609, 10.087332001191507)
(4.0233803082774395, 4.350698340731607)


## 区間推定の結果解釈

1. 真の母集団分布から標本を抽出する
2. 今回と同じやり方で95%信頼区間を計算
3. この試行を繰り返す
4. 全ての試行のうち、真の母数が信頼区間に含まれている割合が95%になる


In [35]:
N = 20000
be_included_array = np.zeros(N, dtype='bool')
# 95%信頼区間を求める試行を 20000 回繰り返す
np.random.seed(1)
norm_dist = stats.norm(loc=4, scale=0.8)
for i in range(0, N):
    sample = norm_dist.rvs(size=10)
    df = len(sample) -1
    mu = sp.mean(sample)
    std = sp.std(sample, ddof=1)
    se = std / sp.sqrt(len(sample))
    interval = stats.t.interval(0.95, df=df, loc=mu, scale=se)
    if interval[0] <= 4 <= interval[1]:
        be_included_array[i] = True

sum(be_included_array) / len(be_included_array)

0.948